In [31]:
from transformers import BertTokenizer, BertModel
import torch
from scipy.spatial.distance import cosine

def get_bert_embedding(sentence, model, tokenizer):
    """Generate BERT embeddings for a given sentence."""
    # Tokenize the sentence and convert to tensor
    input_ids = tokenizer.encode(sentence, return_tensors='pt', add_special_tokens=True, max_length=512, truncation=True)
    with torch.no_grad():
        # Generate embeddings for the sentence
        outputs = model(input_ids)
    # Use the average of all token embeddings as the sentence representation
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def calculate_similarity(embedding1, embedding2):
    """Calculate cosine similarity between two embeddings."""
    # Cosine similarity calculation
    similarity = 1 - cosine(embedding1.numpy(), embedding2.numpy())
    return similarity

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
import pandas as pd

In [12]:
prob_description = pd.read_json('../data/final-prob-desc.json')

In [13]:
prob_description

,id,desc
0,3103. Find Trending Hashtags II,Table: Tweets\n\n+-------------+---------+\n| ...
1,3102. Minimize Manhattan Distances,You are given a 0-indexed array points represe...
2,3101. Count Alternating Subarrays,You are given a binary array nums.\n\nWe call ...
3,3100. Water Bottles II,You are given two integers numBottles and numE...
4,3099. Harshad Number,An integer divisible by the sum of its digits ...
...,...,...
3098,5. Longest Palindromic Substring,"Given a string s, return the longest palindrom..."
3099,4. Median of Two Sorted Arrays,Given two sorted arrays nums1 and nums2 of siz...
3100,3. Longest Substring Without Repeating Characters,"Given a string s, find the length of the longe..."
3101,2. Add Two Numbers,You are given two non-empty linked lists repre...


In [14]:
prod_id = prob_description['id'].unique()
prod_id.sort()

In [15]:
import pandas as pd
import numpy as np

# Create a random matrix or any data you wish to use, ensuring the dimensions match the number of labels
data = np.zeros((len(prod_id), len(prod_id)))

# Create the DataFrame with the same labels for rows and columns
df = pd.DataFrame(data, index=prod_id, columns=prod_id)

In [25]:
import math
import traceback

In [71]:
prob_data = pd.read_csv('../data/final-problem-data.csv')

In [98]:
prob_data['topics'][prob_data['id'] == '10. Regular Expression Matching'].item()

'String,Dynamic Programming,Recursion'

In [102]:
import re
embed_token = pd.DataFrame(index=prod_id, columns=['token'])

for i in range(len(prod_id)):
    sentence = prob_description[prob_description['id'] == prod_id[i]]['desc']
    sentence = sentence.iloc[0]
    try:
        if sentence is not None:
            index = sentence.find('Example 1')
            if index == -1:
                desc = sentence
            else:
                desc = sentence[0:index]
            desc = re.sub("\\n", ' ', desc)
            desc = re.sub("\s+", " ", desc) 
            topics =prob_data['topics'][prob_data['id'] == prod_id[i]]
            if not topics.empty and type(topics.item()) == str:
                desc = desc + " topics: [" + topics.item() + "]"                

            token = get_bert_embedding(desc, model, tokenizer)
            embed_token.loc[prod_id[i]] = [token[0]]
        else:
            embed_token.loc[prod_id[i]] = math.nan
            
    except Exception as err:
        print(prod_id[i])
        print(err)
        print(traceback.format_exc())
        break
        

In [101]:
type('ab') == str

True

In [103]:
for i in range(len(prod_id)):
    embed_1 = embed_token.loc[prod_id[i]][0]
    if embed_1 is math.nan:
        continue

    for j in range(i, len(prod_id)):
        embed_2 = embed_token.loc[prod_id[j]][0]
        if embed_2 is math.nan:
            continue
        simi = calculate_similarity(embed_1, embed_2)
        df.loc[prod_id[i]].loc[prod_id[j]] = simi
        df.loc[prod_id[j]].loc[prod_id[i]] = simi

        

In [6]:
import pandas as pd
df = pd.read_csv('../data/cos-sim-desc-and-topics.csv').set_index('Unnamed: 0')

In [7]:
df

,1. Two Sum,10. Regular Expression Matching,100. Same Tree,1000. Minimum Cost to Merge Stones,1001. Grid Illumination,1002. Find Common Characters,1003. Check If Word Is Valid After Substitutions,1004. Max Consecutive Ones III,1005. Maximize Sum Of Array After K Negations,1006. Clumsy Factorial,...,990. Satisfiability of Equality Equations,991. Broken Calculator,992. Subarrays with K Different Integers,993. Cousins in Binary Tree,994. Rotting Oranges,995. Minimum Number of K Consecutive Bit Flips,996. Number of Squareful Arrays,997. Find the Town Judge,998. Maximum Binary Tree II,999. Available Captures for Rook
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1. Two Sum,1.000000,0.895608,0.0,0.888748,0.864339,0.927714,0.883680,0.932714,0.941823,0.0,...,0.918070,0.912370,0.914368,0.861026,0.893720,0.921225,0.910534,0.834872,0.885692,0.767589
10. Regular Expression Matching,0.895608,1.000000,0.0,0.840534,0.848167,0.912762,0.912532,0.907472,0.900435,0.0,...,0.924634,0.872154,0.869125,0.846004,0.850359,0.895335,0.889189,0.837857,0.896396,0.766480
100. Same Tree,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1000. Minimum Cost to Merge Stones,0.888748,0.840534,0.0,1.000000,0.884365,0.840507,0.831024,0.877274,0.903316,0.0,...,0.868253,0.892897,0.898991,0.845793,0.905554,0.897370,0.886814,0.864439,0.873303,0.866757
1001. Grid Illumination,0.864339,0.848167,0.0,0.884365,1.000000,0.828887,0.843550,0.850859,0.881308,0.0,...,0.878815,0.891041,0.890252,0.857257,0.909108,0.887779,0.872419,0.891561,0.899063,0.853852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995. Minimum Number of K Consecutive Bit Flips,0.921225,0.895335,0.0,0.897370,0.887779,0.890102,0.894897,0.942432,0.948633,0.0,...,0.900891,0.922578,0.942147,0.876943,0.905513,1.000000,0.931069,0.849734,0.915021,0.802484
996. Number of Squareful Arrays,0.910534,0.889189,0.0,0.886814,0.872419,0.869704,0.888505,0.929309,0.926274,0.0,...,0.909331,0.893504,0.941777,0.866009,0.856419,0.931069,1.000000,0.826660,0.904263,0.773799
997. Find the Town Judge,0.834872,0.837857,0.0,0.864439,0.891561,0.803245,0.840025,0.808819,0.852248,0.0,...,0.859747,0.860112,0.841439,0.850852,0.882097,0.849734,0.826660,1.000000,0.881366,0.849263


In [13]:
next = df['285. Inorder Successor in BST'].nlargest(5)

for i, v in next.items():
    print(f"Index: {i}, Value: {v}")

Index: 285. Inorder Successor in BST, Value: 1.0
Index: 1602. Find Nearest Right Node in Binary Tree, Value: 0.9642816185951232
Index: 1448. Count Good Nodes in Binary Tree, Value: 0.9606204032897948
Index: 700. Search in a Binary Search Tree, Value: 0.9605814814567566
Index: 508. Most Frequent Subtree Sum, Value: 0.9605518579483032


In [ ]:
[
    {
        "name" : "Because you solved Content ",
        "list" : [1, 2 ,3]
    },
    {
        "name" : "beca ...",
        "list" : [4, 5, 6]
    }
]

In [17]:
if "sane" == "sane":
    print("macha")

macha


In [19]:
df = pd.read_csv('../data/cos-sim-desc-and-topics.csv').set_index('Unnamed: 0')

In [22]:
df.columns

Index(['1. Two Sum', '10. Regular Expression Matching', '100. Same Tree',
       '1000. Minimum Cost to Merge Stones', '1001. Grid Illumination',
       '1002. Find Common Characters',
       '1003. Check If Word Is Valid After Substitutions',
       '1004. Max Consecutive Ones III',
       '1005. Maximize Sum Of Array After K Negations',
       '1006. Clumsy Factorial',
       ...
       '990. Satisfiability of Equality Equations', '991. Broken Calculator',
       '992. Subarrays with K Different Integers',
       '993. Cousins in Binary Tree', '994. Rotting Oranges',
       '995. Minimum Number of K Consecutive Bit Flips',
       '996. Number of Squareful Arrays', '997. Find the Town Judge',
       '998. Maximum Binary Tree II', '999. Available Captures for Rook'],
      dtype='object', length=3103)

In [30]:
import re
new_index = []
map = {}
for col in df.columns:
    id = re.sub(r'^\d+\.\s*', '', col)
    id = id.lower()
    id = id.strip()
    id = id.replace(" ", "-")
    map[col] = id
    new_index.append(id)

In [49]:
import numpy as np
data = np.zeros((len(new_index), len(new_index)))
new_df = pd.DataFrame(data,columns=new_index, index=new_index)

In [56]:
for col in df.columns:
    new_df[map[col]] = df[col].to_numpy()

In [52]:
new_df['two-sum'] = df['1. Two Sum'].to_numpy()

In [57]:
new_df

,two-sum,regular-expression-matching,same-tree,minimum-cost-to-merge-stones,grid-illumination,find-common-characters,check-if-word-is-valid-after-substitutions,max-consecutive-ones-iii,maximize-sum-of-array-after-k-negations,clumsy-factorial,...,satisfiability-of-equality-equations,broken-calculator,subarrays-with-k-different-integers,cousins-in-binary-tree,rotting-oranges,minimum-number-of-k-consecutive-bit-flips,number-of-squareful-arrays,find-the-town-judge,maximum-binary-tree-ii,available-captures-for-rook
two-sum,1.000000,0.895608,0.0,0.888748,0.864339,0.927714,0.883680,0.932714,0.941823,0.0,...,0.918070,0.912370,0.914368,0.861026,0.893720,0.921225,0.910534,0.834872,0.885692,0.767589
regular-expression-matching,0.895608,1.000000,0.0,0.840534,0.848167,0.912762,0.912532,0.907472,0.900435,0.0,...,0.924634,0.872154,0.869125,0.846004,0.850359,0.895335,0.889189,0.837857,0.896396,0.766480
same-tree,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
minimum-cost-to-merge-stones,0.888748,0.840534,0.0,1.000000,0.884365,0.840507,0.831024,0.877274,0.903316,0.0,...,0.868253,0.892897,0.898991,0.845793,0.905554,0.897370,0.886814,0.864439,0.873303,0.866757
grid-illumination,0.864339,0.848167,0.0,0.884365,1.000000,0.828887,0.843550,0.850859,0.881308,0.0,...,0.878815,0.891041,0.890252,0.857257,0.909108,0.887779,0.872419,0.891561,0.899063,0.853852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
minimum-number-of-k-consecutive-bit-flips,0.921225,0.895335,0.0,0.897370,0.887779,0.890102,0.894897,0.942432,0.948633,0.0,...,0.900891,0.922578,0.942147,0.876943,0.905513,1.000000,0.931069,0.849734,0.915021,0.802484
number-of-squareful-arrays,0.910534,0.889189,0.0,0.886814,0.872419,0.869704,0.888505,0.929309,0.926274,0.0,...,0.909331,0.893504,0.941777,0.866009,0.856419,0.931069,1.000000,0.826660,0.904263,0.773799
find-the-town-judge,0.834872,0.837857,0.0,0.864439,0.891561,0.803245,0.840025,0.808819,0.852248,0.0,...,0.859747,0.860112,0.841439,0.850852,0.882097,0.849734,0.826660,1.000000,0.881366,0.849263
maximum-binary-tree-ii,0.885692,0.896396,0.0,0.873303,0.899063,0.850834,0.915702,0.879805,0.910571,0.0,...,0.898291,0.883450,0.896083,0.886525,0.868519,0.915021,0.904263,0.881366,1.000000,0.794289


In [59]:
new_df.to_csv('../data/content-similarity.csv')

In [4]:
import pandas as pd
df = pd.read_csv('../data/content-similarity.csv').set_index('Unnamed: 0')

In [5]:
df

,two-sum,regular-expression-matching,same-tree,minimum-cost-to-merge-stones,grid-illumination,find-common-characters,check-if-word-is-valid-after-substitutions,max-consecutive-ones-iii,maximize-sum-of-array-after-k-negations,clumsy-factorial,...,satisfiability-of-equality-equations,broken-calculator,subarrays-with-k-different-integers,cousins-in-binary-tree,rotting-oranges,minimum-number-of-k-consecutive-bit-flips,number-of-squareful-arrays,find-the-town-judge,maximum-binary-tree-ii,available-captures-for-rook
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
two-sum,1.000000,0.895608,0.0,0.888748,0.864339,0.927714,0.883680,0.932714,0.941823,0.0,...,0.918070,0.912370,0.914368,0.861026,0.893720,0.921225,0.910534,0.834872,0.885692,0.767589
regular-expression-matching,0.895608,1.000000,0.0,0.840534,0.848167,0.912762,0.912532,0.907472,0.900435,0.0,...,0.924634,0.872154,0.869125,0.846004,0.850359,0.895335,0.889189,0.837857,0.896396,0.766480
same-tree,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
minimum-cost-to-merge-stones,0.888748,0.840534,0.0,1.000000,0.884365,0.840507,0.831024,0.877274,0.903316,0.0,...,0.868253,0.892897,0.898991,0.845793,0.905554,0.897370,0.886814,0.864439,0.873303,0.866757
grid-illumination,0.864339,0.848167,0.0,0.884365,1.000000,0.828887,0.843550,0.850859,0.881308,0.0,...,0.878815,0.891041,0.890252,0.857257,0.909108,0.887779,0.872419,0.891561,0.899063,0.853852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
minimum-number-of-k-consecutive-bit-flips,0.921225,0.895335,0.0,0.897370,0.887779,0.890102,0.894897,0.942432,0.948633,0.0,...,0.900891,0.922578,0.942147,0.876943,0.905513,1.000000,0.931069,0.849734,0.915021,0.802484
number-of-squareful-arrays,0.910534,0.889189,0.0,0.886814,0.872419,0.869704,0.888505,0.929309,0.926274,0.0,...,0.909331,0.893504,0.941777,0.866009,0.856419,0.931069,1.000000,0.826660,0.904263,0.773799
find-the-town-judge,0.834872,0.837857,0.0,0.864439,0.891561,0.803245,0.840025,0.808819,0.852248,0.0,...,0.859747,0.860112,0.841439,0.850852,0.882097,0.849734,0.826660,1.000000,0.881366,0.849263


In [9]:
df['grid-illumination'] + df['two-sum']

Unnamed: 0
two-sum                                      1.864339
regular-expression-matching                  1.743775
same-tree                                    0.000000
minimum-cost-to-merge-stones                 1.773114
grid-illumination                            1.864339
                                               ...   
minimum-number-of-k-consecutive-bit-flips    1.809004
number-of-squareful-arrays                   1.782953
find-the-town-judge                          1.726433
maximum-binary-tree-ii                       1.784755
available-captures-for-rook                  1.621441
Length: 3103, dtype: float64

In [6]:
df['grid-illumination']

Unnamed: 0
two-sum                                      0.864339
regular-expression-matching                  0.848167
same-tree                                    0.000000
minimum-cost-to-merge-stones                 0.884365
grid-illumination                            1.000000
                                               ...   
minimum-number-of-k-consecutive-bit-flips    0.887779
number-of-squareful-arrays                   0.872419
find-the-town-judge                          0.891561
maximum-binary-tree-ii                       0.899063
available-captures-for-rook                  0.853852
Name: grid-illumination, Length: 3103, dtype: float64

In [7]:
df['two-sum']

Unnamed: 0
two-sum                                      1.000000
regular-expression-matching                  0.895608
same-tree                                    0.000000
minimum-cost-to-merge-stones                 0.888748
grid-illumination                            0.864339
                                               ...   
minimum-number-of-k-consecutive-bit-flips    0.921225
number-of-squareful-arrays                   0.910534
find-the-town-judge                          0.834872
maximum-binary-tree-ii                       0.885692
available-captures-for-rook                  0.767589
Name: two-sum, Length: 3103, dtype: float64

In [13]:
last_5 = ['two-sum', 'grid-illumination']
sum = df[last_5[0]]
count = 1
for i in range(1, len(last_5)):
    if last_5[i] in df:
        sum += df[last_5[i]]
        count += 1

sum /= count 

In [15]:
sum.nlargest(10)

Unnamed: 0
grid-illumination                                    1.432170
plates-between-candles                               1.395622
mark-elements-on-array-by-performing-queries         1.394351
minimum-edge-weight-equilibrium-queries-in-a-tree    1.392595
cycle-length-queries-in-a-tree                       1.390480
prison-cells-after-n-days                            1.389458
earliest-second-to-mark-indices-i                    1.388731
sliding-window-median                                1.388575
number-of-islands-ii                                 1.388159
find-maximal-uncovered-ranges                        1.387342
Name: two-sum, dtype: float64